<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [65]:
!pip3 install beautifulsoup4
!pip3 install requests

In [66]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [67]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [68]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/91.0.4472.124 Safari/537.36"
}

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [79]:
import requests

# Define the Wikipedia URL (Falcon 9 and Falcon Heavy launches)
static_url = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"

# Use a browser-like User-Agent to avoid 403
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122 Safari/537.36"
}

# Send HTTP GET
response = requests.get(static_url, headers=headers)
print("Status Code:", response.status_code)   # Expect 200


Status Code: 200


Create a `BeautifulSoup` object from the HTML `response`


In [80]:
from bs4 import BeautifulSoup

# First try with the default parser
soup = BeautifulSoup(response.text, "html.parser")

# Quick check: how many 'wikitable' tables are visible?
tables_htmlparser = soup.find_all("table", {"class": "wikitable"})
print("Tables found with html.parser:", len(tables_htmlparser))

# If none found (or too few), retry with html5lib for robustness
if len(tables_htmlparser) == 0:
    from bs4 import BeautifulSoup as BS
    soup = BS(response.text, "html5lib")
    print("Rebuilt soup with html5lib.")
    print("Tables found with html5lib:", len(soup.find_all('table', {'class': 'wikitable'})))


Tables found with html.parser: 5


Print the page title to verify if the `BeautifulSoup` object was created properly 


In [81]:
print("Page Title:", soup.title.string)


Page Title: List of Falcon 9 and Falcon Heavy launches - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [82]:
# Find all tables that carry the 'wikitable' class
html_tables = soup.find_all("table", {"class": "wikitable"})
print("Total 'wikitable' tables:", len(html_tables))


Total 'wikitable' tables: 5


Starting from the third table is our target table contains the actual launch records.


In [83]:
# The first big launch table is typically at index 2, but this can change. Adjust if needed.
first_launch_table = html_tables[2] if len(html_tables) > 2 else html_tables[0]

# Show the first header row to confirm <th> labels
print(first_launch_table.find("tr"))


<tr>
<th scope="col">Date and time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>booster</a><sup class="reference" id="cite_ref-booster_31-2"><a href="#cite_note-booster-31"><span class="cite-bracket">[</span>i<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_32-2"><a href="#cite_note-Dragon-32"><span class="cite-bracket">[</span>j<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th></tr>


You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [86]:
import re

def extract_column_from_header(th_tag):
    text = th_tag.get_text(" ", strip=True)
    # Remove wiki footnotes like [1], [a]
    text = re.sub(r"\[\s*.*?\s*\]", "", text)
    # Normalize spaces and NBSPs
    text = text.replace("\xa0", " ").strip()
    return text

header_row = first_launch_table.find("tr")
th_tags = header_row.find_all("th")
column_names = [extract_column_from_header(th) for th in th_tags]


Check the extracted column names


In [87]:
print(column_names)

['Date and time ( UTC )', 'Version, booster', 'Launch site', 'Payload', 'Orbit', 'Customer']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [93]:
# Use explicit column names to match the lab rubric
cols = [
    'Flight No.', 'Date', 'Time', 'Version Booster', 'Launch site',
    'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Booster landing'
]
launch_dict = {k: [] for k in cols}


Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [94]:
# Fallback helpers if your lab helpers are not present
import re

def _dt(cell):
    # Best-effort split of date/time from the first column
    raw = cell.get_text(" ", strip=True)
    parts = raw.split()
    date = " ".join(parts[:3]) if len(parts) >= 3 else raw
    time = None
    for p in parts:
        if re.match(r"^\d{2}:\d{2}", p):
            time = p[:5]
            break
    return [date, time]

def _mass(cell):
    txt = cell.get_text(" ", strip=True)
    m = re.search(r"([\d,.\s]+)\s*kg", txt, flags=re.I)
    if not m:
        # Sometimes cell starts with number only
        m = re.search(r"^[\s]*([\d,.\s]+)\b", txt)
    if m:
        val = m.group(1).replace(",", "").strip()
        try:
            return float(val)
        except:
            return None
    return None

extracted_rows = 0

# Loop through every 'wikitable' and parse rows with numeric Flight No.
for table in html_tables:
    for tr in table.find_all("tr"):
        th = tr.find("th")
        if not th or not th.text.strip().isdigit():
            continue

        tds = tr.find_all("td")
        if len(tds) < 9:         # guard: ensure enough cells
            continue

        # 0 Flight No.
        flight_no = th.text.strip()

        # 1 Date & Time
        dt = _dt(tds[0])
        date, time = (dt[0], dt[1])

        # 2 Version Booster
        bv = tds[1].get_text(" ", strip=True)

        # 3 Launch site
        site = tds[2].get_text(" ", strip=True)

        # 4 Payload
        payload = tds[3].get_text(" ", strip=True)

        # 5 Payload mass
        payload_mass = _mass(tds[4])

        # 6 Orbit
        orbit = tds[5].get_text(" ", strip=True)

        # 7 Customer
        customer = tds[6].get_text(" ", strip=True)

        # 8 Launch outcome
        outcome = tds[7].get_text(" ", strip=True)

        # 9 Booster landing (if present)
        landing = tds[8].get_text(" ", strip=True) if len(tds) > 8 else None

        # Append to dict
        launch_dict['Flight No.'].append(flight_no)
        launch_dict['Date'].append(date)
        launch_dict['Time'].append(time)
        launch_dict['Version Booster'].append(bv)
        launch_dict['Launch site'].append(site)
        launch_dict['Payload'].append(payload)
        launch_dict['Payload mass'].append(payload_mass)
        launch_dict['Orbit'].append(orbit)
        launch_dict['Customer'].append(customer)
        launch_dict['Launch outcome'].append(outcome)
        launch_dict['Booster landing'].append(landing)

        extracted_rows += 1

print("Rows parsed:", extracted_rows)
for k, v in launch_dict.items():
    print(f"{k}: {len(v)}")


Rows parsed: 257
Flight No.: 257
Date: 257
Time: 257
Version Booster: 257
Launch site: 257
Payload: 257
Payload mass: 257
Orbit: 257
Customer: 257
Launch outcome: 257
Booster landing: 257


After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [95]:
import pandas as pd

# Build the dataframe directly from lists (safer than dict->Series for mixed lengths)
df = pd.DataFrame(launch_dict)

# Basic cleanup / types
df['Payload mass'] = pd.to_numeric(df['Payload mass'], errors='coerce')

print(df.shape)
display(df.head())


(257, 11)


,Flight No.,Date,Time,Version Booster,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Booster landing
0,286,"January 3, 2024",03:44,F9 B5 B1082‑1,"Vandenberg , SLC‑4E",Starlink : Group 7-9 (22 satellites),16800.0,LEO,SpaceX,Success,Success ( OCISLY )
1,287,"January 3, 2024",23:04,F9 B5 B1076‑10,"Cape Canaveral , SLC‑40",Ovzon-3,1800.0,GTO,Ovzon,Success,Success ( LZ‑1 )
2,288,"January 7, 2024",22:35,F9 B5 B1067‑16,"Cape Canaveral , SLC‑40",Starlink : Group 6-35 (23 satellites),17100.0,LEO,SpaceX,Success,Success ( ASOG )
3,289,"January 14, 2024",08:59,F9 B5 B1061‑18,"Vandenberg , SLC‑4E",Starlink : Group 7-10 (22 satellites),16700.0,LEO,SpaceX,Success,Success ( OCISLY )
4,290,"January 15, 2024",01:52,F9 B5 B1073‑12,"Cape Canaveral , SLC‑40",Starlink : Group 6-37 (23 satellites),17100.0,LEO,SpaceX,Success,Success ( ASOG )


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


In [96]:
df.to_csv('spacex_web_scraped.csv', index=False)

## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
